In [ ]:
import pandas as pd
import numpy as np 
import matplotlib #draw graph
import matplotlib.pyplot as plt
import matplotlib.colors as colors
#downsample data
from sklearn.utils import resample
#split to X y
from sklearn.model_selection import train_test_split 

#warnings
from sklearn.exceptions import ConvergenceWarning

#draws a confusion matrix
from sklearn.metrics import confusion_matrix # from sklearn.metrics import plot_confusion_matrix (not working anymore)
from sklearn.metrics import classification_report, confusion_matrix

#seaborn
import seaborn as sns
from sklearn.svm import SVC 
#nav b
from sklearn.naive_bayes import GaussianNB 

from sklearn.model_selection import train_test_split, GridSearchCV  # Import GridSearchCV

# KNN
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error
from math import sqrt

# Class balancing and data manipulating

In [ ]:
df = pd.read_csv(r'/Users/dt/3920 project/export_merge_res_rodents.csv')

#dropping unwanted attr
# spliting data to smaller scale
df_mh = df[df['Borough'] == 'Manhattan']
df_brx = df[df['Borough'] == 'Bronx']
df_q = df[df['Borough'] == 'Queens']
df_bk = df[df['Borough'] == 'Brooklyn']
df_si = df[df['Borough'] == 'Staten Island']

# seperate data by borough for class balancing
df_manhattan_reshape = df_mh.sample(n=2000, replace=False, random_state=42)
df_queens_reshape = df_q.sample(n=2000, replace=False, random_state=42)
df_bronx_reshape = df_brx.sample(n=2000, replace=False, random_state=42)
df_brooklyn_reshape = df_bk.sample(n=2000, replace=False, random_state=42)
df_state_island_reshape = df_si.sample(n=2000, replace=False, random_state=42)

#combine data 
newdf = pd.concat([df_manhattan_reshape,df_brooklyn_reshape,df_state_island_reshape,df_bronx_reshape,df_queens_reshape], sort = False)
newdf2 = pd.concat([df_manhattan_reshape,df_brooklyn_reshape,df_state_island_reshape,df_bronx_reshape,df_queens_reshape], sort = False)

#replace attr to int
newdf['Borough'].replace(['Manhattan','Queens','Brooklyn','Bronx','Staten Island'],[1,2,3,4,5], inplace = True)
newdf['Grade'].replace(['A','B','C'],[1,2,3],inplace = True)
newdf['Result'].replace([0,1,2,3,4,5,6],[0,1,1,1,1,1,1],inplace = True)

newdf2['Borough'].replace(['Manhattan','Queens','Brooklyn','Bronx','Staten Island'],[1,2,3,4,5], inplace = True)
newdf2['Grade'].replace(['A','B','C'],[1,2,3],inplace = True)
newdf2['Result'].replace([0,1,2,3,4,5,6],[0,1,1,1,1,1,1],inplace = True)


newdf.drop(labels = ['Latitude','Longitude','Inspection_date','Approved_date','Grade_Date','Job_progress','Grade','Census_Tract','Borough',
       'Community_Board'],
           axis = 1,inplace = True)
newdf2.drop(labels = ['Latitude','Longitude','Inspection_date','Approved_date','Grade_Date'],
           axis = 1,inplace = True)

switch str data to int

# Data Exploration

In [ ]:
newdf['Result'].value_counts()

In [ ]:
sns.countplot(x='Result',data=newdf2,palette='hls')
plt.show()
plt.savefig('Result')

In [ ]:
count_rat = len(newdf2[newdf['Result']==1])
count_norat = len(newdf2[newdf['Result']==0])
percent_of_rat = count_rat/(count_rat+count_norat)
print('percentage of no rat is: ',percent_of_rat)
percent_of_no_rat= count_norat/(count_rat+count_norat)
print('percentage of no rat is: ',percent_of_no_rat)

In [ ]:
newdf2.groupby('Result').mean()

-The average grade of restaurant of rat observe is higher than no rat.
-Job Progress around 1.15 have higher changes of rat existence
-Inspection type above 1.19 have higher changes of rat existence

In [ ]:
newdf2.groupby('Inspection_type').mean()

In [ ]:
pd.crosstab(newdf2.Grade,newdf2.Result).plot(kind='bar')
plt.title('Grade with rat result')
plt.xlabel('Grade')
plt.ylabel('Result counts')
plt.savefig('Grade_fre')

In [ ]:
table=pd.crosstab(newdf2.Inspection_type,newdf2.Result)
table.div(table.sum(1).astype(float),axis=0).plot(kind='bar',stacked="True")

In [ ]:
y = newdf2.loc[:, 'Result'] 
X = newdf2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [46]:
X_train

,Inspection_type,Job_progress,Zip_code,Borough,Result,Council_District,Census_Tract,Grade,Community_Board
46763,1.0,1.0,10314.0,5,0.0,51.0,27702.0,2,502.0
53289,2.0,2.0,10312.0,5,1.0,51.0,15601.0,3,503.0
35347,1.0,1.0,10468.0,4,0.0,14.0,23703.0,1,205.0
5091,2.0,2.0,11103.0,2,1.0,22.0,6502.0,3,401.0
38478,1.0,1.0,10003.0,1,0.0,2.0,3602.0,1,103.0
...,...,...,...,...,...,...,...,...,...
41775,2.0,2.0,10303.0,5,1.0,49.0,30302.0,1,501.0
32031,1.0,1.0,10306.0,5,0.0,50.0,13204.0,2,503.0
38674,1.0,1.0,10314.0,5,1.0,51.0,27702.0,1,502.0
37480,2.0,2.0,10002.0,1,0.0,1.0,3601.0,1,103.0


In [ ]:
# naiva bayes 
# X y with 9 features TEST SET (x not included 'result')

clf = GaussianNB()
clf.fit(X_train,y_train)
clfs = clf.predict(X_test)
print(confusion_matrix(y_test,clfs2))
print(classification_report(y_test,clfs2))
X_train.shape

In [48]:
clf2 = KNeighborsClassifier(n_neighbors=325)
clf2.fit(X_train,y_train)
clfs2 = clf2.predict(X_test)
print(confusion_matrix(y_test,clfs2))
print(classification_report(y_test,clfs2))
X_train.shape

[[ 634  479]
 [ 361 1026]]
              precision    recall  f1-score   support

         0.0       0.64      0.57      0.60      1113
         1.0       0.68      0.74      0.71      1387

    accuracy                           0.66      2500
   macro avg       0.66      0.65      0.66      2500
weighted avg       0.66      0.66      0.66      2500



(7500, 9)

In [49]:
clf3 = SVC(random_state=42,probability = True)
clf3.fit(X_train,y_train)
clfs3=clf3.predict(X_test)
print(confusion_matrix(y_test,clfs3))
print(classification_report(y_test,clfs3))

[[ 284  829]
 [ 176 1211]]
              precision    recall  f1-score   support

         0.0       0.62      0.26      0.36      1113
         1.0       0.59      0.87      0.71      1387

    accuracy                           0.60      2500
   macro avg       0.61      0.56      0.53      2500
weighted avg       0.60      0.60      0.55      2500



# Simple Model Emsemble

In [51]:
from itertools import product
from sklearn.ensemble import VotingClassifier

clf = GaussianNB()
clf2 = KNeighborsClassifier(n_neighbors=325)
clf3 = SVC(random_state=42,probability = True)

clf = clf.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)

eclf = VotingClassifier(estimators=[('gnb', clf), ('knn', clf2), ('svc', clf3)],
                         voting='soft', weights=[2, 1, 2])

eclf = eclf.fit(X_train, y_train)

y_pred = eclf.predict(X_test)


print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# Precision = ratio of correctly predicted positive observe
# Recall = ratio of correctly predicted positive observation too all the actual class
# F1 = Weighted average of precision
# Support = number of data used (7500train, 2500 test)
# Accuary = ration of correctly predicted 

[[1067   46]
 [ 340 1047]]
              precision    recall  f1-score   support

         0.0       0.76      0.96      0.85      1113
         1.0       0.96      0.75      0.84      1387

    accuracy                           0.85      2500
   macro avg       0.86      0.86      0.85      2500
weighted avg       0.87      0.85      0.85      2500



In [52]:
x = 'yes'
while x != 'stop':
    print('Inspection type : Inital = 1, Compliance = 2, bait = 3, clean up = 4, stoppage = 5')
    inspec = int(input('Enter inspection type : ')) 
    print('-' * 50)
    
    print('Job progress : Initial = 1, Compliance = 2, bait = 3, clean up = 4, STOPPAGE = 5')
    job_p = int(input('Enter job Progress : '))
    print('-' * 50)
    
    print('''Zip code : (nyc only)''')
    zip_c = int(input('Enter zip code :'))
    print('-' * 50)
    
    print('Borough : Manhattan = 1, Bronx = 2, Brooklyn = 3, Queens = 4, Staten Island = 5')
    bour = int(input('Enter borough :'))
    print('-' * 50)
    
    print('Result : Tract of rat infesation = 1, None = 0' )
    res = int(input('Enter Result code :'))
    print('-' * 50)
    
    print('Council District Code(intergers only)')
    cd = int(input('Enter council district: '))
    print('-' * 50)
    
    print('Census Tract Code(intergers only)')
    ct = int(input('Enter census tract code: '))
    print('-' * 50)
    
    print('Restaurant Grade : A = 1, B = 2, C = 3 ')
    r_grade = int(input('Enter restaurant grade: '))
    print('-' * 50)
    
    print('Community Board (intergers only)')
    cb = int(input('Enter community board: '))
    print('-' * 50)
    
    
    single_data_point2 = [inspec,job_p,zip_c,bour,res,cd,ct,r_grade,cb]
    

    # Reshape the single data point to a 2D array
    reshaped_data_point2 = [single_data_point2]

    # Predictions and probabilities for the single data point
    prediction = eclf.predict(reshaped_data_point2)
    probability = eclf.predict_proba(reshaped_data_point2)

    print("Single Data Point:",single_data_point2)
    print("Prediction:", prediction)
    print("Probabilities:", probability)

    
    x = input('enter STOP end program or anything to continue.')


Inspection type : Inital = 1, Compliance = 2, bait = 3, clean up = 4, stoppage = 5
Enter inspection type : 1
--------------------------------------------------
Job progress : Initial = 1, Compliance = 2, bait = 3, clean up = 4, STOPPAGE = 5
Enter job Progress : 2
--------------------------------------------------
Zip code : (nyc only)
Enter zip code :10003
--------------------------------------------------
Borough : Manhattan = 1, Bronx = 2, Brooklyn = 3, Queens = 4, Staten Island = 5
Enter borough :1
--------------------------------------------------
Result : Tract of rat infesation = 1, None = 0
Enter Result code :0
--------------------------------------------------
Council District Code(intergers only)
Enter council district: 3
--------------------------------------------------
Census Tract Code(intergers only)
Enter census tract code: 61
--------------------------------------------------
Restaurant Grade : A = 1, B = 2, C = 3 
Enter restaurant grade: 1
-----------------------------

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature 

--------------------------------------------------
Single Data Point: [1, 2, 10003, 1, 0, 3, 61, 1, 2]
Prediction: [0.]
Probabilities: [[0.75841628 0.24158372]]
enter STOP end program or anything to continue.stop
